# Descargar automáticamente del SMN
A partir de la ejecución del siguiente código: 
1. Se descargan los archivos de radiación del Servicio Meteorológico Nacional de las fechas seleccionadas.
2. Se estandarizan los mismos y compila en una única tabla.
3. Se los exporta como archivo CSV.

Fuente: https://www.smn.gob.ar/descarga-de-datos

### R        Datos de Radiación de Buenos Aires y de Ushuaia
Datos que se obtienen mediante la medición de la radiación solar y su interacción con la atmósfera.

In [2]:
#Cargar librerías
import requests
import pandas as pd
from datetime import datetime, date, timedelta
from os import  mkdir, rmdir, remove

In [3]:
# Descarga de datos
dir1 = "radiacion" #carpeta de destino de los archivos
mkdir(str(dir1)) 

sdate = date(2019, 1, 1)   # fecha de inicio (año, mes, día)
edate = date(2019, 1, 7)   # fecha de fin (año, mes, día)

delta = edate - sdate

for i in range(delta.days + 1):
    date = str(sdate + timedelta(days=i)).replace('-', '')
    url = 'https://ssl.smn.gob.ar/dpd/descarga_opendata.php?file=radiacionsolar/radsolar' + date + '.txt'
    r = requests.get(url, allow_redirects=True)
    if 'El archivo no existe.' in str(r.content):
        print(date + " does not exist")
    else:
        with open('./' + dir1 + '/' + date + '.txt', 'wb') as f: f.write(r.content)

In [4]:
#Combinación de archivos
out1 = pd.DataFrame(columns = ['Horario', 'Global_BsAs[W/m2]', 'Difusa_BsAs[W/m2]', 'Global_Ush[W/m2]', 'Difusa_Ush[W/m2]'])

for i in range(delta.days + 1):
    date = str(sdate + timedelta(days=i)).replace('-', '')
    df2 = pd.read_csv('./' + dir1 + '/' + date + '.txt', header = None, index_col = False, skiprows= 1,
                    names = ['Horario', 'Global_BsAs[W/m2]', 'Difusa_BsAs[W/m2]', 'Global_Ush[W/m2]', 'Difusa_Ush[W/m2]'])
    out1 = out1.append(df2, ignore_index=True)
    
display(out1.head())
display(out1.tail())

,Horario,Global_BsAs[W/m2],Difusa_BsAs[W/m2],Global_Ush[W/m2],Difusa_Ush[W/m2]
0,2019-01-01 00:01:00,-2.349,-2.187,39.558,35.414
1,2019-01-01 00:02:00,-2.349,-1.988,38.956,33.758
2,2019-01-01 00:03:00,-2.349,-2.584,37.349,31.847
3,2019-01-01 00:04:00,-2.349,-2.584,36.345,30.573
4,2019-01-01 00:05:00,-2.349,-2.584,36.145,29.682


,Horario,Global_BsAs[W/m2],Difusa_BsAs[W/m2],Global_Ush[W/m2],Difusa_Ush[W/m2]
10061,2019-01-07 23:54:00,-3.064,-3.380,17.068,14.268
10062,2019-01-07 23:55:00,-3.064,-3.380,16.265,13.631
10063,2019-01-07 23:56:00,-2.758,-3.380,15.663,13.121
10064,2019-01-07 23:57:00,-2.758,-3.380,15.060,12.611
10065,2019-01-07 23:58:00,-2.758,-3.181,14.659,12.102


In [5]:
#Estandarización
df = out1

df['Horario'] = pd.to_datetime(df['Horario'], format ='%Y-%m-%d %H:%M:%S') 

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10066 entries, 0 to 10065
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Horario            10066 non-null  datetime64[ns]
 1   Global_BsAs[W/m2]  10066 non-null  float64       
 2   Difusa_BsAs[W/m2]  10066 non-null  float64       
 3   Global_Ush[W/m2]   10066 non-null  float64       
 4   Difusa_Ush[W/m2]   10066 non-null  float64       
dtypes: datetime64[ns](1), float64(4)
memory usage: 393.3 KB
None


In [6]:
#Borra archivos anteriores y exporta la tabla
file1 = 'datosRadiacion.csv' #nombre del archivo de destino

df.to_csv(file1, index = False) 

for i in range(delta.days + 1):
    date = str(sdate + timedelta(days=i)).replace('-', '')
    remove('./' + dir1 + '/' + date + '.txt')
rmdir(str(dir1))